In [3]:
from astrometry import astrometry

In [6]:
astro = astrometry("./12_411/data/", "/12_411/data/O20260109_1151.fits")

FileNotFoundError: [Errno 2] No such file or directory: '/12_411/data/O20260109_1151.fits'